# Fundamental Data Extraction
***
Because with Alpha Vantage we don't have access to the complete history of fundamental data of Microsoft, in this notebook I develop code to extract the data directly from Microsoft's investor relations page.

In [55]:
# ------------------------------------------------------------------------------
# LIBRARIES
# ------------------------------------------------------------------------------

# Basic libraries
from tqdm.notebook import tqdm
import warnings
import json

# Data libraries
import pandas as pd
from datetime import datetime

# Visualization tools
import plotly.io as pio

# Personal libraries/modules
%load_ext autoreload
%autoreload 2
import data_processing as dp
import plotting_tools as pt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Libraries configurations
custom_template = pt.generate_plotly_transparent_template(
    color_sequence=pt.CYBERPUNK_COLOR_SEQUENCE,
    title_font_color=pt.FONT_COLOR,
    axis_font_color=pt.FONT_COLOR,
    axis_tick_color=pt.FONT_COLOR,
    axis_grid_color=pt.AXIS_COLOR,
    legend_font_color=pt.FONT_COLOR,
    plot_width=pt.WIDTH,
    plot_height=pt.HEIGHT
)
pio.templates.default = custom_template
pd.options.plotting.backend = 'plotly'
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

# Filter out future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [26]:
# table lists to store quarterly data
is_df_list = []
bs_df_list = []
cf_df_list = []

# Year quarter tuples
year_quarter_list = [
    (year, quarter) for year in range(2009, 2025) for quarter in range(1, 5)
]

# Cycle through years and quarters
for (year, quarter) in tqdm(year_quarter_list):

    # Skip 2009 to Q3: before the data comes in a different format
    if year == 2009 and quarter < 3:
        continue
    
    # Generate the extraction URL
    is_url = f'https://www.microsoft.com/en-us/Investor/earnings/FY-{year}-Q{quarter}/income-statements'
    bs_url = f'https://www.microsoft.com/en-us/Investor/earnings/FY-{year}-Q{quarter}/balance-sheets'
    cf_url = f'https://www.microsoft.com/en-us/Investor/earnings/FY-{year}-Q{quarter}/cash-flows'
    
    # Determine the amount of columns selected
    if year <= 2010 and quarter <= 3:
        cols = [0, 2, 3]
    else:
        cols = [0, 2]
        
    # Income Statements table
    # -----------------------

    # Load table as data frame
    is_df = pd.read_html(is_url)[0]

    # Clean account description column
    account_name = is_df.columns[0]
    is_df[account_name] = (
        is_df.iloc[:, 0].apply(dp.clean_account) + '_' +
        is_df.iloc[:, 1].apply(dp.clean_account)
    )

    # Select current period
    is_df = is_df.iloc[:, cols]

    # Parse period date
    col_names = [
        dp.tuple_to_datetime(t) if i > 0 else 'account' 
        for i, t in enumerate(is_df.columns)
        
    ]

    # Give columns correct names
    is_df.columns = col_names
    value_cols = col_names[1:]

    # Parse numeric data
    is_df[value_cols] = is_df[value_cols].apply(dp.clean_values_series)

    # Parse account data
    is_df['account'] = is_df['account'].str.replace('us-gaap:', '')

    # Drop empty value rows
    is_df = is_df.dropna(axis=0, subset=value_cols)

    # Save results
    is_df_list.append(is_df.set_index('account').T)


    # Balance Sheets table
    # --------------------

    # Load table as data frame
    bs_df = pd.read_html(bs_url)[0]

    # Clean account description column
    account_name = bs_df.columns[0]
    bs_df[account_name] = (
        bs_df.iloc[:, 0].apply(dp.clean_account) + '_' +
        bs_df.iloc[:, 1].apply(dp.clean_account)
    )

    # Select current period
    bs_df = bs_df.iloc[:, cols]

    # Parse period date
    try:
        col_names = [
            datetime.strptime(
                t[1].split(' (')[0].replace(' ', ''), '%B%d,%Y'
            ) 
            if i > 0 else 'account' for i, t in enumerate(bs_df.columns)
        ]
    except:
        col_names = [
            datetime.strptime(
                ' '.join(t).replace(' ', ''), '%B%d,%Y'
            ) 
            if i > 0 else 'account' for i, t in enumerate(bs_df.columns)
        ]


    # Give columns correct names
    bs_df.columns = col_names
    value_cols = col_names[1:]

    # Parse numeric data
    bs_df[value_cols] = bs_df[value_cols].apply(dp.clean_values_series)

        # Parse account data
    bs_df['account'] = bs_df['account'].str.replace('us-gaap:', '')
    # Drop empty value rows
    bs_df = bs_df.dropna(axis=0, subset=value_cols)

    # Save results
    bs_df_list.append(bs_df.set_index('account').T)


    # Cash Flows Statements table
    # ---------------------------

    # Load table as data frame
    cf_df = pd.read_html(cf_url)[0]

    # Clean account description column
    account_name = cf_df.columns[0]
    cf_df[account_name] = (
        cf_df.iloc[:, 0].apply(dp.clean_account) + '_' +
        cf_df.iloc[:, 1].apply(dp.clean_account)
    )

    # Select current period
    cf_df = cf_df.iloc[:, cols]

    # Parse period date
    col_names = [
        dp.tuple_to_datetime(t) if i > 0 else 'account' 
        for i, t in enumerate(cf_df.columns)
        
    ]

    # Give columns correct names
    cf_df.columns = col_names
    value_cols = col_names[1:]

    # Parse numeric data
    cf_df[value_cols] = cf_df[value_cols].apply(dp.clean_values_series)

        # Parse account data
    cf_df['account'] = cf_df['account'].str.replace('us-gaap:', '')
    # Drop empty value rows
    cf_df = cf_df.dropna(axis=0, subset=value_cols)

    # Save results
    cf_df_list.append(cf_df.set_index('account').T)

    # Break if last period reached
    if year == 2024 and quarter == 1:
        break

  0%|          | 0/64 [00:00<?, ?it/s]

In [51]:
# ------------------------------------------------------------------------------
# CONSOLIDATE ALL FUNDAMENTAL DATA
# ------------------------------------------------------------------------------

# Income statements
total_is_df = pd.concat(is_df_list).sort_index().reset_index()
total_is_df = (
    total_is_df
    .groupby('index', as_index=False)
    .apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
    .set_index('index')
    .drop_duplicates()
)

# Balance sheets
total_bs_df = pd.concat(bs_df_list).sort_index().reset_index()
total_bs_df = (
    total_bs_df
    .groupby('index', as_index=False)
    .apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
    .set_index('index')
    .drop_duplicates()
)

# Cash flows
total_cf_df = pd.concat(cf_df_list).sort_index().reset_index()
total_cf_df = (
    total_cf_df
    .groupby('index', as_index=False)
    .apply(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
    .set_index('index')
    .drop_duplicates()
)

# Merge all data
fundamental_df = pd.concat([total_is_df, total_bs_df, total_cf_df], axis=1)

# Drop empty rows
fundamental_df = fundamental_df.dropna(axis=0, how='all')

# Drop empty columns
threshold = int(len(fundamental_df) * 0.9) 
fundamental_df = fundamental_df.dropna(axis=1, thresh=threshold)

# Drop duplicate columns
fundamental_df = fundamental_df.T.drop_duplicates().T

In [53]:
# Generate a mapping from long explanation to simple name
fundamental_data_mapping_dict = dict()
is_count = 1
bs_count = 1
cf_count = 1

for col in fundamental_df.columns:
    if col in total_is_df.columns:
        fundamental_data_mapping_dict[col] = f'is_{is_count}'
        is_count += 1
    elif col in total_bs_df.columns:
        fundamental_data_mapping_dict[col] = f'bs_{bs_count}'
        bs_count += 1
    elif col in total_cf_df.columns:
        fundamental_data_mapping_dict[col] = f'cf_{cf_count}'
        cf_count += 1
    else:
        raise Exception('Column not found in base data')

# Rename columns
fundamental_df = fundamental_df.rename(columns=fundamental_data_mapping_dict)

# Save results in parquet format to avoid re-running process
fundamental_df.to_parquet('data/fundamental_data.parquet')

In [58]:
# Save mapping dict
with open('data/fundamental_variable_mapping.json', 'w') as fp:
    json.dump(fundamental_data_mapping_dict, fp)